In [1]:
library(lme4)
library(ggplot2)
library(here)
library(tidyverse)
library(mixedup)
library(kableExtra)

Loading required package: Matrix

here() starts at /home/sand94/projects/mclark-mixed-models

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ tidyr::pack()   masks Matrix::pack()
✖ tidyr::unpack() masks Matrix::unpack()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘kableExtra’


The following object is masked from ‘package:dplyr’:

    group_rows




In [3]:
kable_df <- function(data, digits=3, ...) {
  kableExtra::kable(
    data,
    digits = digits,
    format = 'html',
    booktabs = T,
    # longtable = F,
    linesep = "", 
    ...,
  ) %>% 
    kableExtra::kable_styling(full_width = F)
}

In [4]:
library(showtext)

font_add_google("Lato", "lato")
showtext_opts(dpi = 300)
showtext_auto()

Loading required package: sysfonts

Loading required package: showtextdb



In [5]:
library(repr)
options(repr.plot.width=15, repr.plot.height=12)

In [6]:
data_location <- "data"
gpa_file <- "gpa.csv"
gpa_data <- read_csv(here(data_location, gpa_file))

Rows: 1200 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): occas, job, sex, admitted
dbl (6): student, gpa, highgpa, year, semester, occasion

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [9]:
gpa_mixed <- lmer(gpa ~ occasion + (1 + occasion | student), data = gpa_data)
summary(gpa_mixed)

Linear mixed model fit by REML ['lmerMod']
Formula: gpa ~ occasion + (1 + occasion | student)
   Data: gpa_data

REML criterion at convergence: 261

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.2695 -0.5377 -0.0128  0.5326  3.1939 

Random effects:
 Groups   Name        Variance Std.Dev. Corr 
 student  (Intercept) 0.045193 0.21259       
          occasion    0.004504 0.06711  -0.10
 Residual             0.042388 0.20588       
Number of obs: 1200, groups:  student, 200

Fixed effects:
            Estimate Std. Error t value
(Intercept) 2.599214   0.018357  141.59
occasion    0.106314   0.005885   18.07

Correlation of Fixed Effects:
         (Intr)
occasion -0.345

### Categorical Features

Sometimes we may want to assess the effects of a categorical feature across the levels of our grouping variable. For our GPA example, this might include something like the binary indicator of whether they took any higher level/AP/Honor courses that semester. Another would be their working status, such as part-time, full-time, volunteer or not working. For many students, this sort of thing could vary from one semester to the next.

For binary features, nothing changes from what we’ve demonstrated so far. We allow that coefficient to vary across groups, just as we would the single coefficient from a numeric predictor. But what about the case where we have additional categories?

As a fixed effect, a categorical feature would have k-1 coefficients with default dummy coding, which represent the effect of changing from a reference group, whose mean is represented via the intercept, to the given category. If we allow that effect to be random, then we would have separate k-1 slopes to vary by our structured levels (along with our random intercept), resulting in multiple random coefficients, along with their correlation.

Adding these additional random slopes would obviously complicate our model, but perhaps not too much. On the practical side though, it can often lead to convergence problems. But there is a silver lining in that case. Since it is a categorical feature, we can just treat it like we would a typical random effect. The one difference is that, in the general case of random coefficients, just as we are interested in the interaction of the effect with structure already present, we will want to specify our model to capture this.

For our example we will run three models:

    Model 1: The way our intuition would suggest based on what we’ve seen so far
    Model 2: An alternate way to code model 1, but which puts our random effects on similar scales
    Model 3: A different, but identically conceptual way to get at our effect via an interaction random effect.

In [14]:
data_location <- "data"
pupil_file <- "pupils.RData"
load(here(data_location, pupil_file))


In [16]:
ach_cat_re_1 <- lmer(
  achievement ~ ses + (1 + ses | primary_school_id),
  data = pupils
)

ach_cat_re_2 <- lmer(
  achievement ~ ses + (0 + ses | primary_school_id),
  data = pupils
)

ach_cat_re_3 <- lmer(
  achievement ~ ses + (1 | primary_school_id) + (1 | primary_school_id:ses),
  data = pupils
)



boundary (singular) fit: see help('isSingular')

boundary (singular) fit: see help('isSingular')



In [17]:
summarize_model(ach_cat_re_1, ci = FALSE, cor_re = TRUE)


Variance Components:




             Group     Effect Variance   SD Var_prop
 primary_school_id  Intercept     0.09 0.30     0.07
 primary_school_id       ses2     0.04 0.20     0.03
 primary_school_id       ses3     0.14 0.37     0.11
 primary_school_id       ses4     0.09 0.30     0.07
 primary_school_id       ses5     0.14 0.37     0.10
 primary_school_id seshighest     0.31 0.55     0.23
          Residual         NA     0.51 0.71     0.39



Fixed Effects:




       Term Value   SE     t P_value Lower_2.5 Upper_97.5
  Intercept  6.08 0.11 57.86    0.00      5.87       6.28
       ses2  0.08 0.13  0.65    0.51     -0.16       0.33
       ses3  0.09 0.13  0.69    0.49     -0.16       0.33
       ses4  0.28 0.12  2.44    0.01      0.06       0.51
       ses5  0.34 0.12  2.92    0.00      0.11       0.57
 seshighest  0.60 0.14  4.33    0.00      0.33       0.88
